# Report
## Project 3: Collaboration and Competition
### Udacity Deep Reinforcement Learning Nanodegree
by Kitson Swann

## Tennis Environment

![playing.gif](playing.gif)

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1. If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01. Thus, the goal of each agent is to keep the ball in play.

### States

The state space consists of 8 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation. 

### Actions

Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping.

# Rewards

The task is episodic. The task is considered solved when the maximum of the total score for the two agents is +0.5 (over 100 consecutive episodes).

After each episode, we add up the rewards that each agent received (without discounting, to get a score for each agent. This yields 2 (potentially different) scores. We then take the maximum of these 2 scores.
This yields a single score for each episode.
The environment is considered solved, when the average (over 100 episodes) of those scores is at least +0.5.

## Agent Version 1

For this project I had planned to initially begin with the simple single agent environment and move onto the more complex multi-agent environment after solving the simple one, but achieving the minimum average score on the single agent version proved quite difficult, so I stopped at solving just the simple environment. Below you can see the agent initialized with the state and action spaces shown.

In [1]:
from unityagents import UnityEnvironment
import matplotlib.pyplot as plt
#from optimize import find_optimal_hyperparameters
from train import train_ddpg

RANDOM_SEED = 2

%matplotlib inline

env = UnityEnvironment(file_name='./env/Tennis.app', seed=RANDOM_SEED, no_graphics=True)

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.29742813 -1.5
 -0.          0.          7.17024279  6.         -0.          0.        ]


In [3]:
import numpy as np

for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.10000000149011612
Score (max over agents) from episode 5: 0.0


In [2]:
env.close()

## Solution

### Algorithm

As suggested in the project instructions I used the Deep Deterministic Policy Gradient agent from the Pendulum environment example [here](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum) as a starting point. I modified the training loop to work with the Unity Reacher environment, which required some minor changes from the original which was based on an OpenAI Gym environment.

The Deep Deterministic Policy Gradient (DDPG) algorithm is similar to a Deep Q Network (DQN) but adapted and modified to suit the task of solving environments with continuous action spaces.

The DDPG algorithm is similar to a DQN in that:
- It uses a replay buffer to store experience tuples which it samples mini-batches from to perform gradient descent steps on the network. 
- It also makes use of local and a target networks to make target values more stationary while training.

However, DQN estimates a Q function that outputs a value for each possible discrete action. Then the agent greedily chooses the action with the highest value. The DDPG instead includes:
- an actor - which is a deterministic policy network that takes in a state and returns an action to take directly and,
- a critic - a value network which takes in the state plus the action from the actor, and returns the value of the state/action pair

In the learning step in DDPG:

- A minibatch is sampled from the replay buffer
- The target actor network predicts the next actions from the states
- The target critic predicts the values of the chosen actions
- The Q targets are calculated
- The Q expected values are calculated using the local critic
- The critic loss is calculated between the expected and target Q values
- The loss is minimized on the local critic network
- The local actor network predicts the next actions
- The actor loss is calculated using the local critic
- The loss is minimized on the local actor network
- The actor and critic target networks are soft updated with the local network parameters

### Initial Experimentation

After getting the training routine working, I initially had no success in getting the agent to train with the standard hyper-parameters. The score was oscillating around 0-0.1 most of the time and not improving even over 100 episodes.

Normally at this point I would have experimented with hyperparameter-optimization, but in this algorithm there are so many parameters that I felt the need to look for some good values. I consulted Udacity's Knowledge Base for some assistance, and saw a number of suggestions. I reviewed the suggestions and many repositories containing the solutions of other students for ideas. I made a number of changes as a result including:

- Changing the training loop to end when done which for the Reacher environment is 1000 timesteps.
- Trying different random seeds.
- Modified the architecture of the actor and critic neural networks to try different hidden layer sizes.
- Modified the architecture of the actor and critic neural networks to add batch normalization.
- Modified the DDPG agent to only do a learning step every 5, 10, 20, 40 iterations.
- Reset the noise after every episode.
- Tried different suggested values for buffer size, batch size, gamma, tau, weight decay and the sigma variable in the Ornstein-Uhlenbeck process noise implementation.

Even with all of the above modifications I still wasn't seeing any decent results. The score rarely went above 0.5, and seemed to get worse over time.

### Structured Hyper-parameter Optimization

After seeing a number of people say the suggested changes should work, but having no success myself, I felt that I needed to take a more robust approach to hyper-parameter optimization. Similar to my implementation in Project 1, I used scikit-optimize's Gaussian Process Optimizer to search for a better set of hyper-parameters. Below is the search space that I initially defined.

I initially did batches of 10 episodes to keep the run time short and look for some better initial values. Because the process takes a long time I watched the values from each run, and visually tried to understand what values of each parameter were working well. As I noticed good values, I iteratively removed parameters from the search space, instead setting them at reasonable levels until I found something that worked fairly well which I've called Reasonable Parameter Set below.

I have included the call to the optimization routine below, but haven't run it as it takes up a lot of space, and I re-ran it iteratively so one single output was never the final one used.

In [ ]:
from skopt.space import Real, Integer, Categorical

space = [
    Real(0, 0.5, "uniform", name='eps_end'),
    Real(1e-5, 1e0, "log-uniform", name='eps_decay'),
    Categorical([1e5, 1e6], name="buffer_size"),
    Categorical([64, 128, 256, 512], name="batch_size"),
    Categorical([1, 5, 10, 20, 40], name="update_every"),
    Categorical([1, 5, 10, 20], name="update_times"),
    Categorical([128, 256, 512], name="actor_fc1_units"),
    Categorical([128, 256, 512], name="actor_fc2_units"),
    Categorical([128, 256, 512], name="critic_fc1_units"),
    Categorical([128, 256, 512], name="critic_fc2_units"),
    Real(1e-5, 1e-3, "log-uniform", name='actor_lr'),
    Real(1e-5, 1e-3, "log-uniform", name='critic_lr'),
    Real(0.9, 0.99, "uniform", name="gamma"),
    Real(1e-2, 1e-1, "log-uniform", name="tau"),
    Real(0, 1e-3, "uniform", name="weight_decay"),
    Real(0.01, 0.05, "log-uniform", name="noise_theta"),
    Real(0.01, 0.05, "log-uniform", name="noise_sigma"),
]

params, res_gp = find_optimal_hyperparameters(env=env, brain_name=brain_name, num_agents=num_agents, n_calls=50, episodes_per_batch=10, space=space)

#### Reasonable Parameter Set

This is the best resonable parameter set I found through a somewhat manual search for hyper-parameters. There are many parameters to tune here, and the search space grows so quickly that a full optimization run was going to take more time than I had available, so I viewed the results and manually tuned based on what I saw to develop the set of values below.

In [ ]:
optimal_params = {
  "random_seed": RANDOM_SEED,
  "noise_sigma": 0.02,
  "noise_theta": 0.015,
  "weight_decay": 0,
  "tau": 0.05,
  "gamma": 0.95,
  "critic_lr": 0.0001,
  "actor_lr": 0.001,
  "critic_fc2_units": 128,
  "critic_fc1_units": 128,
  "actor_fc2_units": 128,
  "actor_fc1_units": 128,
  "update_times": 10,
  "update_every": 20,
  "batch_size": 256,
  "buffer_size": 1000000.0,
  "eps_decay": 0.7,
  "eps_end": 0.3,
  "eps_start": 1.0,
  "action_size": action_size,
  "state_size": state_size,
  "n_episodes": 20000,
  "solved_threshold": 0.5,
  "break_early": True,
  "name": "_base",
  "num_agents": 2,
  "brain_name": brain_name
}

scores = train_ddpg(env=env, **optimal_params)

{
  "random_seed": 2,
  "noise_sigma": 0.02,
  "noise_theta": 0.015,
  "weight_decay": 0,
  "tau": 0.05,
  "gamma": 0.95,
  "critic_lr": 0.0001,
  "actor_lr": 0.001,
  "critic_fc2_units": 128,
  "critic_fc1_units": 128,
  "actor_fc2_units": 128,
  "actor_fc1_units": 128,
  "update_times": 10,
  "update_every": 20,
  "batch_size": 256,
  "buffer_size": 1000000.0,
  "eps_decay": 0.7,
  "eps_end": 0.3,
  "eps_start": 1.0,
  "action_size": 2,
  "state_size": 24,
  "n_episodes": 20000,
  "solved_threshold": 0.5,
  "break_early": true,
  "name": "_base",
  "num_agents": 2,
  "brain_name": "TennisBrain"
Replay Buffer Size: 1000000.0, Batch Size: 256
Episode 9	Average Score: 0.0	Score: 0.0

/Users/kitson_swann/Documents/udacity_drl/udacity_drl_p3/agent.py:166: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)


Episode 100	Average Score: 0.0	Max Score: 0.0	Min Score: 0.0
Episode 200	Average Score: 0.0	Max Score: 0.0	Min Score: 0.0
Episode 300	Average Score: 0.0	Max Score: 0.0	Min Score: 0.0
Episode 400	Average Score: 0.0546	Max Score: 0.1	Min Score: 0.1
Episode 500	Average Score: 0.0653	Max Score: 0.1	Min Score: 0.1
Episode 600	Average Score: 0.0764	Max Score: 0.0	Min Score: 0.0
Episode 700	Average Score: 0.0646	Max Score: 0.09	Min Score: 0.09
Episode 800	Average Score: 0.0645	Max Score: 0.1	Min Score: 0.1
Episode 900	Average Score: 0.079	Max Score: 0.1	Min Score: 0.1
Episode 1000	Average Score: 0.0811	Max Score: 0.1	Min Score: 0.1
Episode 1100	Average Score: 0.0837	Max Score: 0.1	Min Score: 0.1
Episode 1200	Average Score: 0.0844	Max Score: 0.0	Min Score: 0.0
Episode 1300	Average Score: 0.0851	Max Score: 0.1	Min Score: 0.1
Episode 1400	Average Score: 0.0888	Max Score: 0.09	Min Score: 0.09
Episode 1500	Average Score: 0.0855	Max Score: 0.0	Min Score: 0.0
Episode 1600	Average Score: 0.0868	Max S

In [ ]:
import json

with open("scores.json", "w") as f:
    json.dump({"scores": scores}, f, indent=2)

### Success !!!

As you can see above the environment was solved in 639 episodes. In reality, it was a bit shorter than that, because I set the threshold to 31 just to go a bit beyond the minimim value.

Below you can see the plots of raw scores and average scores that I took from Tensorboard.

![tensorboard_raw_score.png](tensorboard_raw_score.png)
![tensorboard_avg_score.png](tensorboard_avg_score.png)

## Watch The Trained Agent

Below I've loaded the trained agent and shown a gif of it performing the task.

In [1]:
from unityagents import UnityEnvironment
env = UnityEnvironment(file_name='./env/Tennis.app')

RANDOM_SEED=2
NUM_AGENTS=2

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
from agent import Agent
import torch
import numpy as np

params = {
  "random_seed": RANDOM_SEED,
  "noise_sigma": 0.02,
  "noise_theta": 0.015,
  "weight_decay": 0,
  "tau": 0.05,
  "gamma": 0.95,
  "critic_lr": 0.0001,
  "actor_lr": 0.001,
  "critic_fc2_units": 128,
  "critic_fc1_units": 128,
  "actor_fc2_units": 128,
  "actor_fc1_units": 128,
  "update_times": 10,
  "update_every": 20,
  "batch_size": 256,
  "buffer_size": 1000000.0,
  "eps_decay": 0.7,
  "eps_end": 0.3,
  "eps_start": 1.0,
  "action_size": 2,
  "state_size": 24,
}

agent = Agent(**params)

agent.actor_local.load_state_dict(torch.load('checkpoint_actor_base.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic_base.pth'))
        
env_info = env.reset(train_mode=False)[brain_name]
max_scores = []

for i_episode in range(1000):
    agent.reset()
    states = env_info.vector_observations
    agent_scores = np.zeros(NUM_AGENTS)
    i = 1
    while True:
        actions = agent.act(state=states, add_noise=False)
        actions = np.clip(actions, -1, 1)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        states = next_states
        agent_scores += rewards
        print(f"\rTimestep: {i}\tAgent 1 Reward: {round(rewards[0],4)}\tAgent 2 Reward: {round(rewards[1],4)}", end = "")
        if np.any(dones):
            break
        i += 1
    
    episode_max = np.max(agent_scores)
    print(f"\rEpisode: {i_episode}\tMax Reward: {round(episode_max,4)}\tAgent 1 Total Reward: {round(agent_scores[0],4)}\tAgent 2 Total Reward: {round(agent_scores[1],4)}")

env.close()

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Replay Buffer Size: 1000000.0, Batch Size: 256
Episode: 0	Max Reward: 0.09	Agent 1 Total Reward: 0.0	Agent 2 Total Reward: 0.09
Episode: 1	Max Reward: 0.09	Agent 1 Total Reward: 0.0	Agent 2 Total Reward: 0.09
Episode: 2	Max Reward: 0.1	Agent 1 Total Reward: 0.1	Agent 2 Total Reward: -0.01
Episode: 3	Max Reward: 0.0	Agent 1 Total Reward: 0.0	Agent 2 Total Reward: -0.01
Episode: 4	Max Reward: 0.1	Agent 1 Total Reward: 0.1	Agent 2 Total Reward: -0.01
Episode: 5	Max Reward: 0.1	Agent 1 Total Reward: 0.1	Agent 2 Total Reward: -0.01
Episode: 6	Max Reward: 0.1	Agent 1 Total Reward: 0.1	Agent 2 Total Reward: 0.09
Episode: 7	Max Reward: 0.0	Agent 1 Total Reward: 0.0	Agent 2 Total Reward: -0.01
Episode: 8	Max Reward: 0.0	Agent 1 Total Reward: 0.0	Agent 2 Total Reward: -0.01
Episode: 9	Max Reward: 0.0	Agent 1 Total Reward: 0.0	Agent 2 Total Reward: -0.01
Episode: 10	Max Reward: 0.1	Agent 1 Total Reward: 0.1	Agent 2 Total Reward: -0.01
Episode: 11	Max Reward: 0.09	Agent 1 Total Reward: 0.0	Agent 2

KeyboardInterrupt: 

![playing.gif](playing.gif)

## Longer Training Run with Larger Network

I wanted to see how much better the score could get with a slightly longer training run and networks with larger hidden layers. I also modified the batch size since it seems to have an effect on how long each iteration takes.

Below you can see that performance improved slightly breaking the previous score, but it seemed to be hitting a plateau just above 30, so I terminated it.

After understanding that there are always 1000 time steps in an episode, and the rewards are closer to 0.04 rather than 0.10 as per timestep that the arm is touching the goal, I realized there is an upper limit on the score of 0.40 * 1000 = 40. I think this is why the agent plateues just after a score of 30, because it is getting closer and closer to a perfect performance.

In [ ]:
alternate_params = {
  "random_seed": 22,
  "noise_sigma": 0.02,
  "noise_theta": 0.015,
  "weight_decay": 0,
  "tau": 0.05,
  "gamma": 0.95,
  "critic_lr": 0.0001,
  "actor_lr": 0.001,
  "critic_fc2_units": 300,
  "critic_fc1_units": 400,
  "actor_fc2_units": 300,
  "actor_fc1_units": 400,
  "update_times": 10,
  "update_every": 20,
  "batch_size": 256,
  "buffer_size": 500000.0,
  "eps_decay": 0.7,
  "eps_end": 0.3,
  "eps_start": 1.0,
  "action_size": 4,
  "state_size": 33,
  "n_episodes": 5000,
  "solved_threshold": 31.0,
  "break_early": False,
  "name": "_alternate",
  "num_agents": 1,
  "brain_name": "ReacherBrain"
}

scores = train_ddpg(env=env, **alternate_params)

# Ideas for Future Work

If I had more time and access to a GPU I could conduct a more exhaustive hyper-parameter search, letting the agent run for at least 100 episodes each time and I would probably find a better set of parameters, but I don't think this would lead to a significantly better solution for the reason I mentioned above that there isan upper limit on performance. It may lead to a faster to train and more efficient solution though.

Using the same algorithm, I could also make slight modifications to train with the multi-agent version of the Reacher environment. I did try this but didn't see initially better performance and it seemed that the hyper-parameters were more important. Now that I've found some decent options I could further investigate the multi-agent version.

In the project instructions, there were also number of other algorithms mentioned that may work better including:

- Trust Region Policy Optimization (TRPO)
- Truncated Natural Policy Gradient (TNPG)
- Proximal Policy Optimization (PPO)
- Distributed Distributional Deterministic Policy Gradients (D4PG)

The next steps for me would be to investigate and attempt to solve the task with one or more of these alternate algorithms.